In [1]:
from utils.experimentor import *
from utils.constants import *  

In [2]:
config = {
    "dataset_name": Dataset.OGBN_PROTEINS,
    "model_type": ModelType.GATV1,
    'num_of_epochs': 10000,
    'num_of_runs': 1,
    'patience_period': 100,
    
    #'should_visualize': False,
    'batch_size': 512,
    'test_batch_size': 1024,
    'num_workers': 2,
    'force_cpu': False,
    'test_frequency': 10,
    'console_log_freq': 10,
    
    'lr': 0.001,
    'num_of_layers': 3,  # PPI has got 42% of nodes with all 0 features - that's why 3 layers are useful
    'num_heads': 1,
    'hidden_size': 256,
    #'num_heads_per_layer': [4, 4, 6],  # other values may give even better results from the reported ones
    #'num_features_per_layer': [PPI_NUM_INPUT_FEATURES, 256, 256, PPI_NUM_CLASSES],  # 64 would also give ~0.975 uF1!
    #'add_skip_connection': True,  # skip connection is very important! (keep it otherwise micro-F1 is almost 0)
    #'bias': True,  # bias doesn't matter that much
    'dropout': 0.0,  # dropout hurts the performance (best to keep it at 0)
    
    #'neighbourhood_degree': 2,
    #'adjacency_mode': AdjacencyMode.OneStep
}

In [3]:
experimentor = Experimentor(config)
experimentor.run()

torch.cuda is available: True
Used Dataset: Dataset.OGBN_PROTEINS


C:\Users\doria\anaconda3\envs\PS\lib\site-packages\torch\nn\init.py:388: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


sffj tensor([1., 1., 0.,  ..., 0., 0., 0.], device='cuda:0')

Run 01:



Epoch 01:   0%|          | 0/86619 [00:00<?, ?it/s]

ValueError: Target size (torch.Size([512])) must be the same as input size (torch.Size([512, 1]))